<a href="https://colab.research.google.com/github/hyunlimy/SNUBH_Digital_Health/blob/main/02_lowbp_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 경기도 바이오 전문인력 양성사업 - 디지털 헬스케어 AI 솔루션 개발과 산업현장 체험
## 생체신호 데이터 활용 (1) <br> Low blood pressure prediction using arterial wave - preprocessing
- Author : **Hyun-Lim Yang, Ph.D.**<br>
Research Assistant Professor @
Seoul National University Hospital <br>
Department of Anesthesiology and Pain Medicine
- E-mail : hlyang{_at_}snu{_dot_}ac{_dot_}kr
***

In [ ]:
from IPython.display import HTML
style_warn = "<style>div.warn { background-color: #fcf2f2;border-color: #dFb5b4; border-left: 5px solid #dfb5b4; padding: 0.5em;}</style>"
HTML(style_warn)

### Import packages

<div class="warn">**Warning!** : use your directory at `download_directory`</div>


> **Wargning** <br>
> colab 환경을 위한 google drive import 코드가 포함되어 있습니다. <br>
> 로컬 환경에서 실행 시 colab을 위한 import function들을 comment out 한 뒤 실행하세요.

In [ ]:
from google.colab import drive  # for colab
drive.mount('/content/gdrive/')  # for colab

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import os
cloud_directory = '/content/gdrive/My Drive/SNUBH_Digital_Healthcare/'
os.listdir(cloud_directory) # for colab, check cloud directory mount

['kohi_preprocessor.py', 'kohi_CNN_model_archi.png', 'data']

In [ ]:
import sys
download_directory = os.getcwd() # for local environments
sys.path.append(download_directory) # for local environments
sys.path.append(cloud_directory) # for colab

In [ ]:
!pip install vitaldb

In [ ]:
!pip install gcsfs
import gcsfs

In [ ]:
import numpy as np
import pandas as pd
import glob
import kohi_preprocessor as pre
import vitaldb
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')

download_directory = cloud_directory # for colab

### Data loading
샘플 파일을 vitaldb 서버로 부터 직접 다운받아 로드

> **TODO:** `00010.vital`을 100hz로 변환하여 `chart_pd_01`에 DataFrame으로 저장하기

In [ ]:
track_names = ["SNUADC/ART", "Solar8000/ART_MBP"]
### =========== Your code here ====================

vitalfile = vitaldb.VitalFile(10, track_names=track_names)
chart_pd_01 = vitalfile.to_pandas(track_names, interval=1/100)

### ===============================================


In [ ]:
chart_pd_01

,SNUADC/ART,Solar8000/ART_MBP
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
2099216,2.05234,NaN
2099217,2.05234,NaN
2099218,2.05234,NaN
2099219,2.05234,NaN


In [ ]:
col_mbp = 'Solar8000/ART_MBP'
col_art = 'SNUADC/ART'

### mean blood pressure 데이터 추출
> **TODO:** ART_MBP 변수들 중 값이 존재하는 것만 별도의 변수 `mbp_data_pd`에 저장

In [ ]:
# mbp 데이터 추출
### =========== Your code here ====================

mbp_data_pd = chart_pd_01[col_mbp][chart_pd_01[col_mbp].notnull()]

### ===============================================

mbp_index = mbp_data_pd.index.values
print(mbp_data_pd.head())

708    -20.0
899    -20.0
1099   -20.0
1299   -20.0
1499   -20.0
Name: Solar8000/ART_MBP, dtype: float64


### arterial wave 데이터 추출 및 nan 지우기
> **TODO:** ART 변수들 중 값이 존재하지 않는 모든 부분을 0으로 채운 뒤 `art_full_pd`에 저장

In [ ]:
# art 데이터 전체 추출 및 nan value 채우기
art_full_pd = chart_pd_01[col_art]

### =========== Your code here ====================

art_full_pd = art_full_pd.fillna(0)

### =========== Your code here ====================
print(art_full_pd.head())

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: SNUADC/ART, dtype: float64


### 데이터셋 만들기
필요한 파라미터들 정의

In [ ]:
# 필요한 파라미터들 정의
mdelay = 1
srate = 100
length = 20
max_limit_mbp = 250
min_limit_mbp = 0

In [ ]:
# 입력 데이터 길이가 20s이고 black out (여기서는 mdelay) 기간이 1분 이니, mbp가 1분 20초 이내에 등장하는 것은 무시함
# mbp index를 1분 20초 뒤로 미룸
mbp_points = mbp_index[mbp_index > (mdelay*60*srate + length*srate)]
print(mbp_points)

[   8100    8300    8500 ... 2092959 2093159 2093359]


### Arterial wave segment 추출
> **TODO:** 현재 mbp 값을 기준으로, 1분 20초 전 ~ 1분 전 데이터를 arterial wave segment로 추출하여 `art_seg_list`에 저장하기

In [ ]:
# 실습: arterial segment 추출하기
mbp_values_list = [mbp_data_pd[idx] for idx in mbp_points]

### =========== Your code here ====================

art_seg_list = [art_full_pd[idx-(mdelay*60*srate) - (length*srate) :
                            idx - (mdelay*60*srate)].values
                for idx in mbp_points]
### ===============================================

In [ ]:
mbp_values_np = np.array(mbp_values_list)
art_seg_np = np.array(art_seg_list)
print(mbp_values_np.shape)
print(art_seg_np.shape)

(10428,)
(10428, 2000)


### 조건에 따라 filter들 정의

In [ ]:
# 필터들 선언
# mbp min-max filter
mbp_max_filter = mbp_values_np < max_limit_mbp
mbp_min_filter = mbp_values_np > min_limit_mbp
mbp_filter = mbp_max_filter & mbp_min_filter

# abp range filter
art_filter_list = []
for seg in art_seg_np:
    filter_value = (np.array(seg) > 25.0).all() and (np.array(seg) < 250.0).all()
    art_filter_list.append(filter_value)
art_filter = np.array(art_filter_list)

# mstds 필터
mstds_values_list = []
for seg in tqdm(art_seg_np):
    if (np.array(seg) < 0.).any():
        mstds_values_list.append(float(0.))
    else:
        mstd_val, _ = pre.process_beat(seg)
        mstds_values_list.append(mstd_val)
mstds_filter = np.array(mstds_values_list) > 0.

100%|██████████| 10428/10428 [02:16<00:00, 76.20it/s] 


전체 필터 하나로 만들기

In [ ]:
all_filters = mbp_filter & art_filter & mstds_filter

### 필터 적용하여 데이터 추출
> **TODO:** `all_filters`를 이용하여 필터 통과한 데이터들만 추출해 `mbp_filtered`와 `art_filtered`에 저장

In [ ]:
### =========== Your code here ====================

mbp_filtered = mbp_values_np[all_filters]
art_filtered = art_seg_np[all_filters]

### ===============================================

print(mbp_filtered.shape)
print(art_filtered.shape)

(8104,)
(8104, 2000)


데이터셋 정의

In [ ]:
mbp_filtered[mbp_filtered < 61].shape

(626,)

### Binary label 정의
mbp가 61 미만인 데이터들의 label을 1로 지정하여 학습을 위한 y_label 생성

In [ ]:
label_tmp = np.zeros(len(mbp_filtered))
label_tmp[mbp_filtered<61] = 1.

In [ ]:
x_data = art_filtered
y_label = label_tmp

In [ ]:
x_data.shape

(8104, 2000)

In [ ]:
y_label.shape

(8104,)